In [1]:
## everything working

import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sklearn.metrics.pairwise import cosine_similarity
from trulens_eval import TruChain, Feedback, Tru
import numpy as np

# Initialize Tru
tru = Tru()
# tru.reset_database()  # Uncomment if you want to reset the database

# Load and preprocess the data
loader = TextLoader("D:/TruLens/speech.txt", encoding='utf-8')
documents = loader.load()
print(f"Loaded {len(documents)} documents")

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} text chunks")

# Create embeddings and vector store with matching dimensions
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")  # 768 dimensions
print("Embeddings model loaded")

vectorstore = Chroma.from_documents(texts, embeddings)
print("Vector store created successfully")

# Create a retriever
retriever = vectorstore.as_retriever()

# Set up the language model (FLAN-T5)
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("Language model loaded")

def truncate_text(text, max_tokens=512):
    tokens = tokenizer.encode(text, truncation=False)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
    return tokenizer.decode(tokens)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    device="cpu"  # Use "cuda" if you have a GPU
)

llm = HuggingFacePipeline(pipeline=pipe)
print("Pipeline setup complete")

# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)
print("QA chain created")

# Simplified feedback functions
def simple_relevance(record):
    if isinstance(record, dict):
        question = record.get("input", "")
        answer = record.get("output", "")
    else:
        question = answer = record
    return cosine_similarity([embeddings.embed_query(question)], [embeddings.embed_query(answer)])[0][0]

def simple_groundedness(record):
    if isinstance(record, dict):
        answer = record.get("output", "")
        context = " ".join([doc.page_content for doc in record.get("source_documents", [])])
    else:
        answer = context = record
    answer_tokens = set(answer.lower().split())
    context_tokens = set(context.lower().split())
    return len(answer_tokens.intersection(context_tokens)) / len(answer_tokens) if answer_tokens else 0

# Define feedback for TruLens
f_relevance = Feedback(simple_relevance, name="relevance").on_output()
f_groundedness = Feedback(simple_groundedness, name="groundedness").on_output()

# Ensure app_id matches
app_id = "RAG_QA_Chain"

# Create TruChain
tru_recorder = TruChain(
    qa_chain,
    app_id=app_id,
    feedbacks=[f_relevance, f_groundedness],
)
print("TruChain created")

# Example questions
questions = [
    "What is the amrit kaal?",
    "What cause damage to infrastructure?",
    "What is jai hind?"
]

# Run evaluations and ensure they are being recorded
for question in questions:
    with tru_recorder as recording:
        truncated_question = truncate_text(question)
        response = qa_chain({"query": truncated_question})
    
    print(f"\nQuestion: {question}")
    print(f"Answer: {response['result']}")

# Get records and feedback
records, feedback = tru.get_records_and_feedback(app_ids=[app_id])
print("\nRecords:")
print(records)

# Print column names
print("\nColumn names:")
print(records.columns)

# Print evaluation results
print("\nEvaluation Results:")
for _, record in records.iterrows():
    print(f"Question: {record['input']}")
    print(f"Answer: {record['output']}")
    for column in records.columns:
        if column not in ['input', 'output']:
            print(f"{column}: {record[column]}")
    print("---")

# Launch dashboard
tru.run_dashboard()
print("Dashboard launched successfully")

Created a chunk of size 1243, which is longer than the specified 1000
Created a chunk of size 1042, which is longer than the specified 1000
Created a chunk of size 1428, which is longer than the specified 1000
Created a chunk of size 1277, which is longer than the specified 1000
Created a chunk of size 1196, which is longer than the specified 1000
Created a chunk of size 1283, which is longer than the specified 1000
Created a chunk of size 1323, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of size 1170, which is longer than the specified 1000
Created a chunk of size 1107, which is longer than the specified 1000
Created a chunk of size 1035, which is longer than the specified 1000
Created a chunk of size 1053, which is longer than the specified 1000
Created a chunk of size 1486, which is longer than the specified 1000
Created a chunk of size 1093, which is longer than the specified 1000


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Loaded 1 documents
Split into 67 text chunks
Embeddings model loaded
Vector store created successfully


Feedback implementation <function simple_relevance at 0x0000029EED44E2A0> cannot be serialized: Module __main__ is not importable. This may be ok unless you are using the deferred feedback mode.
Feedback implementation <function simple_groundedness at 0x0000029EED44E3E0> cannot be serialized: Module __main__ is not importable. This may be ok unless you are using the deferred feedback mode.


Language model loaded
Pipeline setup complete
QA chain created
✅ In relevance, input record will be set to __record__.main_output or `Select.RecordOutput` .
✅ In groundedness, input record will be set to __record__.main_output or `Select.RecordOutput` .
TruChain created


Token indices sequence length is longer than the specified maximum sequence length for this model (847 > 512). Running this sequence through the model will result in indexing errors



Question: What is the amrit kaal?
Answer: time of duty

Question: What cause damage to infrastructure?
Answer: bomb blasts

Question: What is jai hind?
Answer: Ministry of Jal Shakti

Records:
          app_id                                           app_json  \
0   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
1   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
2   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
3   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
4   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
5   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
6   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
7   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
8   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
9   RAG_QA_Chain  {"tru_class_info": {"name": "TruChain", "modul...   
10  RAG_QA_Chain  {"tru_c

Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.6:8501 .
Dashboard launched successfully
